In [4]:
import glob, os, json
import numpy as np
import scipy.misc
from PIL import Image
from shutil import copyfile

In [5]:
mvc_info = json.load(open(r"S:\Projects\MVC\mvc_info.json"))

In [ ]:
# Resize

size = 128, 128
total = 161260

for i in range(33820, total):
    path = "S:\\Projects\\MVC\\images\\" + str(i) + ".jpg"
    im = Image.open(path).resize(size)
    im.save("S:\\Projects\\MVC\\128x128\\" + str(i) + ".jpg", "JPEG")

In [ ]:
# Copy

src_path = "S:\\Projects\\MVC\\128x128_clean\\"
dst_path = "H:\\vincentw\\Projects\\CycleGAN-tensorflow\\datasets\\front2left\\"

for i in range(0, 26844, 4):
    copyfile(src_path + str(i) + ".jpg", dst_path + "trainA\\" + str(i) + ".jpg")
    copyfile(src_path + str(i + 1) + ".jpg", dst_path + "trainB\\" + str(i + 1) + ".jpg")

In [7]:
# Clean

counter = 26840
src_path = r"S:\\Projects\\MVC\\128x128\\"
dst_path = r"S:\\Projects\\MVC\\128x128_clean\\"

for i in range(30000, 60000-3):
    if mvc_info[i]['productId'] != mvc_info[i+1]['productId'] or \
    mvc_info[i+1]['productId'] != mvc_info[i+2]['productId'] or \
    mvc_info[i+2]['productId'] != mvc_info[i+3]['productId']:
        continue
    if mvc_info[i]['styleId'] != mvc_info[i+1]['styleId'] or \
    mvc_info[i+1]['styleId'] != mvc_info[i+2]['styleId'] or \
    mvc_info[i+2]['styleId'] != mvc_info[i+3]['styleId']:
        continue
    if mvc_info[i]['colourId'] != mvc_info[i+1]['colourId'] or \
    mvc_info[i+1]['colourId'] != mvc_info[i+2]['colourId'] or \
    mvc_info[i+2]['colourId'] != mvc_info[i+3]['colourId']:
        continue
    if mvc_info[i]['viewId'] == 0 and mvc_info[i+1]['viewId'] == 1 and mvc_info[i+2]['viewId'] == 2 and mvc_info[i+3]['viewId'] == 3:
        #print(i)
        copyfile(src_path + str(i) + ".jpg", dst_path + str(counter) + ".jpg")
        copyfile(src_path + str(i+1) + ".jpg", dst_path + str(counter+1) + ".jpg")
        copyfile(src_path + str(i+2) + ".jpg", dst_path + str(counter+2) + ".jpg")
        copyfile(src_path + str(i+3) + ".jpg", dst_path + str(counter+3) + ".jpg")
        counter = counter + 4

In [ ]:
# Concat

src_path = "S:\\Projects\\MVC\\128x128_clean\\"
dst_path = "S:\\Projects\\MVC\\128x128_concat\\"

for i in range(0, 26844, 4):
    
    path1 = src_path + str(i) + ".jpg"
    path2 = src_path + str(i+1) + ".jpg"
    im1 = np.array(Image.open(path1))
    im2 = np.array(Image.open(path2))
    im = np.concatenate((im1, im2), axis=1)
    scipy.misc.imsave(dst_path + str(i) + ".jpg", im)

In [13]:
# VAE Concat

gt_path = "S:\\Projects\\MVC\\128x128_clean\\"
src_path = "S:\\Projects\\MVC\\128x128_vae\\"
dst_path = "S:\\Projects\\MVC\\128x128_gan\\"

count = 0

for i in range(0, 10000, 4):
    for j in range(4):
        for k in range(4):
            gt_img = np.array(Image.open(gt_path + str(i+j) + ".jpg"))
            src_img = np.array(Image.open(src_path + str(i+k) + "_" + str(j) + ".jpg"))
            im = np.concatenate((src_img, gt_img), axis=1)
            scipy.misc.imsave(dst_path + str(count) + ".jpg", im)
            count += 1

In [11]:
# Test

path = "S:\\Projects\\MVC\\64x64_test\\backup\\"

for file in os.listdir(path):
    im = np.array(Image.open(path + file))
    # A->B
    #im[:,int(im.shape[1]/2):,:] = 255
    # B->A
    im = np.concatenate((np.full((im.shape[0], int(im.shape[1]/2), im.shape[2]), 255), im[:,int(im.shape[1]/2):,:]), axis=1)
    scipy.misc.imsave(path + "_" + file, im)